<a href="https://colab.research.google.com/github/apurba420/Cassava_Leaf_Classification-traing_on_TPU/blob/main/Leaf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fsspec
!pip install gcsfs

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras.regularizers import l2

**Activate TPU accelerator**

In [3]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy

In [4]:
strategy = auto_select_accelerator()

INFO:tensorflow:Initializing the TPU system: grpc://10.34.60.234:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.34.60.234:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Running on TPU: grpc://10.34.60.234:8470
Running on 8 replicas


**Image Decoder Function**

In [5]:
def build_decoder(with_labels=True, target_size=(256, 256), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode

**Data Augmentation Function**

In [6]:
def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        img = tf.image.adjust_saturation(img, 2.2)
        img = tf.image.random_contrast(img, 0.2, 0.4)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment

**Creating Dataset Function**

In [7]:
def build_dataset(paths, labels=None, bsize=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)
    
    return dset

**Google Cloud Bucket Link**

In [8]:
GCS_DS_PATH = 'gs://kds-e118bcdb309cf88b7f9e4a96ee84997123a5781b886180ffc13d3fc9'

**Read the CSV file**


In [9]:
df = pd.read_csv(GCS_DS_PATH +'/train.csv')
sub_df = pd.read_csv(GCS_DS_PATH +'/sample_submission.csv')

In [10]:
df

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3
...,...,...
21392,999068805.jpg,3
21393,999329392.jpg,3
21394,999474432.jpg,1
21395,999616605.jpg,4


In [11]:
paths = GCS_DS_PATH + "/train_images/" + df['image_id']
test_paths = GCS_DS_PATH + "/test_images/" + sub_df['image_id']

In [12]:
labels = df["label"]

In [13]:
labels

0        0
1        3
2        1
3        1
4        3
        ..
21392    3
21393    3
21394    1
21395    4
21396    4
Name: label, Length: 21397, dtype: int64

**Train test split**

In [14]:
(
    train_paths, valid_paths, 
    train_labels, valid_labels
) = train_test_split(paths, labels, test_size=0.2, random_state=42)

In [15]:
train_paths[4], train_labels[4]

('gs://kds-e118bcdb309cf88b7f9e4a96ee84997123a5781b886180ffc13d3fc9/train_images/1000812911.jpg',
 3)

In [16]:
img_size = 256
batch_size = strategy.num_replicas_in_sync * 16

In [17]:
decoder = build_decoder(with_labels=True, target_size=(img_size, img_size))
test_decoder = build_decoder(with_labels=False, target_size=(img_size, img_size))

train_dataset = build_dataset(
    train_paths, train_labels, bsize=batch_size, decode_fn=decoder
)

valid_dataset = build_dataset(
    valid_paths, valid_labels, bsize=batch_size, decode_fn=decoder,
    repeat=False, shuffle=False, augment=False
)

test_dataset = build_dataset(
    test_paths, cache=False, bsize=batch_size, decode_fn=test_decoder,
    repeat=False, shuffle=False, augment=False
)

In [18]:
train_dataset

<PrefetchDataset shapes: ((None, 256, 256, 3), (None,)), types: (tf.float32, tf.int64)>

In [19]:
with strategy.scope():
    model = tf.keras.Sequential([
                                   tf.keras.applications.InceptionResNetV2(
          include_top=False,
      weights="imagenet",
      input_shape=(256,256,3)),
      tf.keras.layers.GlobalAveragePooling2D(),
      tf.keras.layers.Dense(64, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)),
      tf.keras.layers.Dropout(0.5),
      tf.keras.layers.Dense(32, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)),
      tf.keras.layers.Dropout(0.5),
      tf.keras.layers.Dense(5, activation='softmax')
      ])
    model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001), metrics=['acc'])
    model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Functio (None, 6, 6, 1536)        54336736  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1536)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                98368     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 1

In [20]:
steps_per_epoch = train_paths.shape[0] // batch_size
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'Cassava_model.h5', save_best_only=True, monitor='val_loss', mode='min')
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", patience=5, min_lr=1e-6, mode='min')

In [ ]:
history = model.fit(
    train_dataset, 
    epochs=20,
    steps_per_epoch=steps_per_epoch,callbacks = [checkpoint, lr_reducer],
    validation_data=valid_dataset)

Epoch 1/20
 84/133 [=================>............] - ETA: 2:10 - loss: 2.7828 - acc: 0.6008